In [1]:
%cd ../..

c:\Users\HuyLQ15_CTV\Desktop\model_monitoring


In [2]:
from datetime import time, datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
from datetime import datetime, time
from source.mlops_observation import ColumnMapping

content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip", verify=False).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-31 00:00:00']
current = raw_data.loc['2011-02-01 00:00:00':'2011-07-28 23:00:00']
current['time_stamp'] = current.index
reference['time_stamp'] = reference.index
column_mapping.datetime_features = 'time_stamp'

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ho-proxy02.tpb.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Period Data Quality Summary

In [3]:
# import pandas as pd
# from typing import Dict

# def dataset_summary_to_dataframe(dataset: pd.DataFrame, column_mapping: ColumnMapping, period: str='M') -> pd.DataFrame:
#     """
#     Convert a dictionary of DatasetSummary objects into a DataFrame.

#     Parameters:
#     - dataset: pd.DataFrame
#         Dataset to be calculated the summary stats
#     - column_mapping: ColumnMapping
#     - period: str

#     Returns:
#     - pd.DataFrame: A DataFrame with dates as columns and DatasetSummary attributes as index.
#     """
#     dataset_summary_dict = {}             # Dict[str, DatasetSummary]
#     data_period_dict = divide_into_chunks_by_period(dataset, column_mapping.datetime_features, period=period)
#     for data_date in data_period_dict.keys():
#         dataset_summary_dict[data_date] = calculate_dataset_common_stats(data_period_dict[data_date], column_mapping)
#     data = {}

#     for date, summary in dataset_summary_dict.items():
#         # Convert the DatasetSummary object to a dictionary
#         summary_dict = summary.__dict__
        
#         # Remove "number_uniques_by_columns" 
#         summary_dict.pop('number_uniques_by_columns', None)
#         summary_dict.pop('nans_by_columns', None)

#         # Add the data for this date
#         data[date] = summary_dict
#     df = pd.DataFrame(data).reset_index()
#     df.rename(columns={'index': ''}, inplace=True)
#     return df


In [4]:
# df1 = dataset_summary_to_dataframe(current, column_mapping)
# df2 = dataset_summary_to_dataframe(reference, column_mapping)


In [5]:
# pd.merge(df2, df1, on='')

In [6]:
# import plotly.graph_objects as go
# from typing import Optional, Dict, List
# from evidently.model.widget import BaseWidgetInfo
# from evidently.base_metric import MetricResult, Metric, InputData
# from evidently.model.widget import BaseWidgetInfo
# from evidently.base_metric import InputData, Metric, MetricResult
# from evidently import ColumnMapping
# from evidently.renderers.base_renderer import default_renderer
# from evidently.renderers.base_renderer import MetricRenderer
# from evidently.utils.visualizations import plot_distr
# from evidently.renderers import html_widgets
# from evidently.renderers.html_widgets import plotly_data
# from evidently.renderers.html_widgets import header_text
# from source.mlops_observation.utils.visualization import dataframe_to_widget

In [7]:
# class PeriodDataQuality(MetricResult):
#     class Config:
#         type_alias = "evidently:metric_result:PeriodDataQuality"
#     summary_table: Optional[pd.DataFrame]

# class PeriodDataQualityMetric(Metric[PeriodDataQuality]):
#     """
#     time_period (str): Offset aliases https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
#     """
#     class Config:
#         type_alias = "evidently:metric:PeriodDataQualityMetric"
#     _time_period: str
#     def __init__(self, time_period: str = 'M'):
#         self._time_period = time_period
#         super().__init__()

#     def calculate(self, data: InputData):
#         if data.reference_data is None:
#             raise ValueError("Reference dataset should be present")
#         if data.current_data is None:
#             raise ValueError("Current dataset should be present")
#         column_mapping = data.column_mapping
#         if data.current_data is not None:
#             reference_summary_table = self._dataset_summary_to_dataframe(
#                 dataset=data.reference_data,
#                 column_mapping=column_mapping,
#                 period=self._time_period
#             )
#             reference_summary_table.columns = [reference_summary_table.columns[0]] + [f"{col} (base)" for col in reference_summary_table.columns[1:]]
#             current_summary_table = self._dataset_summary_to_dataframe(
#                 dataset=data.current_data,
#                 column_mapping=column_mapping,
#                 period=self._time_period
#             )
#             summary_table = pd.merge(reference_summary_table, current_summary_table, on='')
#         else:
#             summary_table = self._dataset_summary_to_dataframe(
#                 dataset=data.reference_data,
#                 column_mapping=column_mapping,
#                 period=self._time_period
#             )
#         return PeriodDataQuality(
#             summary_table=summary_table
#         )
#     def _dataset_summary_to_dataframe(
#         self, 
#         dataset: pd.DataFrame, 
#         column_mapping: str, 
#         period: str='M'
#         ) -> pd.DataFrame:
#         """
#         Convert a dictionary of DatasetSummary objects into a DataFrame.

#         Parameters:
#         - dataset_summary_dict: A dictionary where keys are dates (str) and values are DatasetSummary objects.

#         Returns:
#         - pd.DataFrame: A DataFrame with dates as columns and DatasetSummary attributes as index.
#         """
#         dataset_summary_dict = {}             # Dict[str, DatasetSummary]
#         data_period_dict = divide_into_chunks_by_period(dataset, column_mapping.datetime_features, period=period)
#         for data_date in data_period_dict.keys():
#             dataset_summary_dict[data_date] = calculate_dataset_common_stats(data_period_dict[data_date], column_mapping)
#         data = {}

#         for date, summary in dataset_summary_dict.items():
#             # Convert the DatasetSummary object to a dictionary
#             summary_dict = summary.__dict__
            
#             # Remove "number_uniques_by_columns" 
#             summary_dict.pop('number_uniques_by_columns', None)
#             summary_dict.pop('nans_by_columns', None)

#             # Add the data for this date
#             data[date] = summary_dict
#         df = pd.DataFrame(data).reset_index()
#         df.rename(columns={'index': ''}, inplace=True)
#         return df

# @default_renderer(wrap_type=PeriodDataQualityMetric)
# class PeriodDataQualityRender(MetricRenderer):
#     def render_html(self, obj: PeriodDataQualityMetric) -> List[BaseWidgetInfo]:
#         results = obj.get_result()
#         color_options = self.color_options
#         summary_table = dataframe_to_widget(results.summary_table)
#         return [
#             header_text(label=f"Data Quality in period Report"),
#             summary_table
#         ]

In [8]:
# from source.mlops_observation import Report
# from source.mlops_observation.metrics import PeriodDataQualityMetric
# report = Report([PeriodDataQualityMetric(time_period='M')])
# report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)
# report.save_html('1.html')

# period feature stats 

In [9]:
from source.mlops_observation.utils.data_operations import divide_into_chunks_by_period

reference_dict = divide_into_chunks_by_period(reference, time_column=column_mapping.datetime_features, period='M')
reference_dict

c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:DatasetUtilityColumns') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")
c:\Users\HuyLQ15_CTV\AppData\Local\miniconda3\envs\model-monitoring\lib\site-packages\evidently\pydantic_utils.py:161: UserWarning: Duplicate key (<class 'evidently.base_metric.MetricResult'>, 'evidently:metric_result:DatasetColumns') in alias map
  warnings.warn(f"Duplicate key {key} in alias map")


{'2011-01':                      instant  season  yr  mnth  hr  holiday  weekday  \
 2011-01-01 00:00:00        1       1   0     1   0        0        6   
 2011-01-01 01:00:00        2       1   0     1   1        0        6   
 2011-01-01 02:00:00        3       1   0     1   2        0        6   
 2011-01-01 03:00:00        4       1   0     1   3        0        6   
 2011-01-01 04:00:00        5       1   0     1   4        0        6   
 ...                      ...     ...  ..   ...  ..      ...      ...   
 2011-01-30 20:00:00      661       1   0     1  20        0        0   
 2011-01-30 21:00:00      662       1   0     1  21        0        0   
 2011-01-30 22:00:00      663       1   0     1  22        0        0   
 2011-01-30 23:00:00      664       1   0     1  23        0        0   
 2011-01-31 00:00:00      665       1   0     1   0        0        1   
 
                      workingday  weathersit  temp   atemp   hum  windspeed  \
 2011-01-01 00:00:00           0

In [10]:
reference

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,time_stamp
2011-01-01 00:00:00,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16,2011-01-01 00:00:00
2011-01-01 01:00:00,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40,2011-01-01 01:00:00
2011-01-01 02:00:00,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32,2011-01-01 02:00:00
2011-01-01 03:00:00,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13,2011-01-01 03:00:00
2011-01-01 04:00:00,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1,2011-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30 20:00:00,661,1,0,1,20,0,0,0,2,0.26,0.2727,0.65,0.1045,3,30,33,2011-01-30 20:00:00
2011-01-30 21:00:00,662,1,0,1,21,0,0,0,2,0.24,0.2424,0.70,0.1642,3,25,28,2011-01-30 21:00:00
2011-01-30 22:00:00,663,1,0,1,22,0,0,0,2,0.24,0.2273,0.70,0.1940,2,19,21,2011-01-30 22:00:00
2011-01-30 23:00:00,664,1,0,1,23,0,0,0,2,0.24,0.2121,0.65,0.2836,5,16,21,2011-01-30 23:00:00


In [11]:
from evidently.calculations.data_quality import get_features_stats
from evidently.core import ColumnType
from evidently.metrics.data_integrity.column_summary_metric import ColumnCharacteristics
from evidently.metrics.data_integrity.column_summary_metric import FeatureQualityStats
from evidently.metrics.data_integrity.column_summary_metric import NumericCharacteristics
from evidently.metrics.data_integrity.column_summary_metric import CategoricalCharacteristics
from evidently.metrics.data_integrity.column_summary_metric import DatetimeCharacteristics


def map_data(stats: FeatureQualityStats) -> ColumnCharacteristics:
    if stats.feature_type == "num":
        if isinstance(stats.max, str) or isinstance(stats.min, str) or isinstance(stats.most_common_value, str):
            raise ValueError("max / min stats should be int or float type, but got str")
        return NumericCharacteristics(
            number_of_rows=stats.number_of_rows,
            count=stats.count,
            mean=stats.mean,
            std=stats.std,
            min=stats.min,
            max=stats.max,
            p25=stats.percentile_25,
            p50=stats.percentile_50,
            p75=stats.percentile_75,
            unique=stats.unique_count,
            unique_percentage=stats.unique_percentage,
            missing=stats.missing_count,
            missing_percentage=stats.missing_percentage,
            infinite_count=stats.infinite_count,
            infinite_percentage=stats.infinite_percentage,
            most_common=stats.most_common_value,
            most_common_percentage=stats.most_common_value_percentage,
        )
    if stats.feature_type == "cat":
        return CategoricalCharacteristics(
            number_of_rows=stats.number_of_rows,
            count=stats.count,
            unique=stats.unique_count,
            unique_percentage=stats.unique_percentage,
            most_common=stats.most_common_value,
            most_common_percentage=stats.most_common_value_percentage,
            missing=stats.missing_count,
            missing_percentage=stats.missing_percentage,
        )
    if stats.feature_type == "datetime":
        if not isinstance(stats.min, str) or not isinstance(stats.max, str):
            raise ValueError(f"min / max expected to be str for datetime, got {type(stats.min)}/{type(stats.max)}")
        return DatetimeCharacteristics(
            number_of_rows=stats.number_of_rows,
            count=stats.count,
            unique=stats.unique_count,
            unique_percentage=stats.unique_percentage,
            most_common=stats.most_common_value,
            most_common_percentage=stats.most_common_value_percentage,
            missing=stats.missing_count,
            missing_percentage=stats.missing_percentage,
            first=stats.min,
            last=stats.max,
        )
    raise ValueError(f"unknown feature type {stats.feature_type}")

In [12]:
from typing import Union
from typing import Dict
def period_characteristic_table(data: Dict[str, Union[CategoricalCharacteristics, NumericCharacteristics]]) -> pd.DataFrame:
    """
    Convert a dictionary of characteristics into a DataFrame for display.
    
    Parameters:
    - data: A dictionary where keys are period labels (e.g., '2011-01') and values are
            characteristics dictionaries (e.g., 'CategoricalCharacteristics' or 'NumericCharacteristics').

    Returns:
    - A DataFrame with periods as columns and characteristics as rows.
    """
    # Initialize an empty DataFrame
    result_df = pd.DataFrame()
    for key in data.keys():
        data[key] = data[key].__dict__

    # Process each period's characteristics
    for period, characteristics in data.items():
        # Format characteristics into a dictionary suitable for DataFrame
        formatted_characteristics = {
            key: (
                f"{value} ({characteristics[f'{key}_percentage']:.2f}%)" 
                if f"{key}_percentage" in characteristics 
                else value
            )
            for key, value in characteristics.items()
            if key not in ['type', 'unique_percentage', 'most_common_percentage', 'missing_percentage', 'infinite_percentage']
        }
        # Add to the result DataFrame
        result_df[period] = pd.Series(formatted_characteristics)

    # Set a proper index (characteristic names) and return
    return result_df

In [13]:
a = map_data(get_features_stats(reference['temp'], ColumnType.Numerical))
period_characteristic_table({'2021-09': a})

,2021-09
number_of_rows,665
count,665
missing,0 (0.00%)
mean,0.2
std,0.08
min,0.02
p25,0.16
p50,0.2
p75,0.24
max,0.46


In [14]:
from typing import Dict
def calculate_characteristic(data_chunks: Dict[str, pd.DataFrame], col_name: str, col_type: str)-> Dict[str, pd.DataFrame]:
    feature_stat_dict = {}
    if col_type == 'num':
        col_type = ColumnType.Numerical
        index_name_mapping = {
            "number_of_rows": "number of rows",
            "count": "count",
            "missing": "missing",
            "mean": "mean",
            "std": "std",
            "min": "min",
            "p25": "25%",
            "p50": "50%",
            "p75": "75%",
            "max": "max",
            "unique": "unique",
            "most_common": "most common",
            "missing_percentage": "missing",
            "infinite_count": "infinite",
            "infinite_percentage": "infinite",
        }
    elif col_type == 'cat':
        col_type = ColumnType.Categorical
        index_name_mapping = {
            "number_of_rows": "count",
            "count": "count",
            "missing": "missing",
            "unique": "unique",
            "most_common": "most common"
        }
    else:
        raise ValueError("col_type must be 'cat' or 'num'")
    for key in data_chunks.keys():
        feature_stat_dict[key] = map_data(get_features_stats(data_chunks[key][col_name], col_type))
    result = period_characteristic_table(feature_stat_dict)
    result.rename(index=index_name_mapping, inplace=True)
    if col_type == ColumnType.Categorical:
        result.drop(['new_in_current_values_count', 'unused_in_current_values_count'], axis=0, inplace=True)
    return result 

In [15]:
cur_chunks_dict = divide_into_chunks_by_period(current, time_column=column_mapping.datetime_features, period='M')
ref_chunks_dict = divide_into_chunks_by_period(reference, time_column=column_mapping.datetime_features, period='M')
ref = calculate_characteristic(ref_chunks_dict, 'temp', 'cat')
cur = calculate_characteristic(cur_chunks_dict, 'temp', 'cat')

In [16]:
pd.merge(left=ref, right=cur, left_index=True, right_index=True) \
    .reset_index() \
    .rename({'index': ''}, axis=1) \
    .drop_duplicates()

,,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07
0,count,665,649,730,719,744,720,672
4,missing,0 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%),0 (0.00%)
5,most common,0.16 (13.83%),0.22 (10.32%),0.34 (11.10%),0.46 (7.65%),0.52 (10.22%),0.7 (10.97%),0.7 (11.61%)
6,unique,23 (3.46%),31 (4.78%),30 (4.11%),27 (3.76%),29 (3.90%),24 (3.33%),20 (2.98%)


In [17]:
# from source.mlops_observation.utils.visualization import dataframe_to_widget
# from source.mlops_observation.utils.data_quality import calculate_characteristic
# from source.mlops_observation.utils.data_operations import divide_into_chunks_by_period
# from evidently.base_metric import Metric
# from evidently.base_metric import MetricResult
# from evidently.base_metric import InputData
# from evidently.renderers.html_widgets import BaseWidgetInfo
# from evidently.renderers.html_widgets import header_text
# from evidently.renderers.base_renderer import MetricRenderer
# from evidently.renderers.base_renderer import default_renderer
# from typing import List

In [18]:
# class PeriodFeatureQualityResult(MetricResult):
#     class Config:
#         type_alias = "evidently:metric_result:PeriodFeatureQualityResult"
#     num_feature_stats: Dict[str, pd.DataFrame]     # {col_name, stats}
#     cat_feature_stats: Dict[str, pd.DataFrame]     # {col_name, stats}

# class PeriodFeatureQualityMetric(Metric[PeriodFeatureQualityResult]):
#     """
#     time_period (str): Offset aliases https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
#     """
#     class Config:
#         type_alias = "evidently:metric:PeriodFeatureQualityMetric"
#     _time_period: str
#     def __init__(self, time_period: str = 'M'):
#         self._time_period = time_period
#         super().__init__()

#     def calculate(self, data: InputData):
#         num_feature_stats = {}
#         cat_feature_stats = {}
#         column_mapping = data.column_mapping
#         if column_mapping.datetime_features is None:
#             raise ValueError('column_mapping.datetime_features can not be None')
#         if data.reference_data is not None:
#             reference_chunks_dict = divide_into_chunks_by_period(data.reference_data, time_column=column_mapping.datetime_features, period=self._time_period)
            
#         current_chunks_dict = divide_into_chunks_by_period(data.current_data, time_column=column_mapping.datetime_features, period=self._time_period)
#         if column_mapping.numerical_features is not None:
#             for num_col in column_mapping.numerical_features:
#                 current_stats = calculate_characteristic(current_chunks_dict, num_col, 'num')
#                 if data.reference_data is not None:
#                     reference_stats = calculate_characteristic(reference_chunks_dict, num_col, 'num')
#                     reference_stats.columns = [f"{col} (base)" for col in reference_stats.columns]
#                     num_feature_stats[num_col] = pd.merge(reference_stats, current_stats, left_index=True, right_index=True) \
#                         .reset_index() \
#                         .rename({'index': ''}, axis=1)
#                 else:
#                     num_feature_stats[num_col] = current_stats \
#                         .reset_index() \
#                         .rename({'index': ''}, axis=1)
        
#         if column_mapping.categorical_features is not None:
#             for cat_col in column_mapping.categorical_features:
#                 current_stats = calculate_characteristic(current_chunks_dict, cat_col, 'cat')
#                 if data.reference_data is not None:
#                     reference_stats = calculate_characteristic(reference_chunks_dict, cat_col, 'cat')
#                     reference_stats.columns = [f"{col} (base)" for col in reference_stats.columns]
#                     cat_feature_stats[cat_col] = pd.merge(reference_stats, current_stats, left_index=True, right_index=True) \
#                         .reset_index() \
#                         .rename({'index': ''}, axis=1) \
#                         .drop_duplicates()
#                 else:
#                     cat_feature_stats[cat_col] = current_stats \
#                         .reset_index() \
#                         .rename({'index': ''}, axis=1) \
#                         .drop_duplicates()
        
#         return PeriodFeatureQualityResult(
#             num_feature_stats=num_feature_stats,
#             cat_feature_stats=cat_feature_stats
#         )


# from evidently.renderers.html_widgets import ColumnDefinition
# from evidently.renderers.html_widgets import RichTableDataRow
# from evidently.renderers.html_widgets import RowDetails
# from evidently.renderers.html_widgets import rich_table_data
# from evidently.renderers.html_widgets import rich_table_data
# from typing import Optional
# @default_renderer(wrap_type=PeriodFeatureQualityMetric)
# class PeriodFeatureQualityRender(MetricRenderer):
#     def _generate_column_params(
#         self,
#         column_data: pd.DataFrame
#         )-> Optional[RichTableDataRow]:
#         details = RowDetails()
#         # -----------------------------------------
#         table = column_data
#         table = dataframe_to_widget(table)
#         details.with_part("STAT TABLE", info=table)
#         # -----------------------------------------
        
#         return RichTableDataRow({
#             "column_name": column_data.col_name, 
#             "column_type": column_data.col_type,
#             "period_drifted_count": fr"{column_data.period_drifted_count}/{column_data.period_count}"
#             }, 
#             details=details)
#     def render_html(self, obj: PeriodFeatureQualityMetric) -> List[BaseWidgetInfo]:
#         results = obj.get_result()
#         color_options = self.color_options
#         rows = []
#         # summary_table = dataframe_to_widget(results.summary_table)
#         for cat_col in results.cat_feature_stats.keys():
#             details = RowDetails()
#             table = dataframe_to_widget(results.cat_feature_stats[cat_col])
#             details.with_part("STAT TABLE", info=table)
#             rows.append(RichTableDataRow({
#                 "column_name": cat_col, 
#                 "column_type": 'cat'}, 
#                 details=details))
#         for num_col in results.num_feature_stats.keys():
#             details = RowDetails()
#             table = dataframe_to_widget(results.num_feature_stats[num_col])
#             details.with_part("STAT TABLE", info=table)
#             rows.append(RichTableDataRow({
#                 "column_name": num_col, 
#                 "column_type": 'num'}, 
#                 details=details))
#         columns = [
#             ColumnDefinition("Column Name", "column_name"),
#             ColumnDefinition("Type", "column_type")
#         ]
#         return [
#             header_text(label=f"Feature Statistic in Period"),
#             rich_table_data(
#                 columns=columns,
#                 data=rows
#             )
#         ]

In [20]:
from source.mlops_observation import Report
from source.mlops_observation.metrics import PeriodDataQualityMetric
from source.mlops_observation.metrics import PeriodDataDriftMetric
from source.mlops_observation.metrics import PeriodFeatureQualityMetric
report = Report([
    PeriodDataDriftMetric(time_period='M'),
    PeriodDataQualityMetric(time_period='M'),
    PeriodFeatureQualityMetric(time_period='M'),
    ])
report.run(reference_data=None, current_data=current, column_mapping=column_mapping)
report.save_html('1.html')